In [2]:
import re 
import os
import csv
import logging
import requests
from bs4 import BeautifulSoup
from bs4 import NavigableString

logging.basicConfig(filename="flats.log",     
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')

In [28]:
logging.info('test')

In [ ]:
url = "https://www.avito.ru/nizhniy_novgorod/kvartiry/sdam/na_dlitelnyy_srok"
response = requests.get(url)
response

In [8]:
response.content[:100]

b'<!DOCTYPE html>\n          \n<html> <head>\n    <script>\n window.dataLayer = [{"pageType":"SearchResult'

In [14]:
tree = BeautifulSoup(response.content, 'html.parser')
flats = tree.find_all('div', {'class' : 'description item_table-description'})
flats[0]

<div class="description item_table-description"> <div class="item_table-header">
<h3 class="title item-description-title" data-marker="item-title"> <a class="item-description-title-link" href="/nizhniy_novgorod/kvartiry/studiya_31.1_m_2324_et._1797799290" itemprop="url" target="_self" title="Студия, 31.1 м², 23/24 эт. в Нижнем Новгороде"> <span itemprop="name">Студия, 31.1 м², 23/24 эт.</span> </a>
</h3>
<div class="about" itemprop="offers" itemscope="" itemtype="http://schema.org/Offer">
<span content="RUB" itemprop="priceCurrency"></span><span class="price price_highlight" content="18300" data-marker="item-price" itemprop="price">
 18 300  <span class="font_arial-rub">₽</span> в месяц  </span> </div>
</div>
<div> <p class="address" data-marker="item-address">
<i class="i-metro i-metro-niz-1" title="Автозаводская линия"></i>
  Чкаловская  <span class="c-2">800 м</span>
  ,    Нижний Новгород, улица Июльских Дней, 1к2  </p> </div>
<div class="data">
                    Агентство    </d

In [31]:
details = flats[0].div
details

<div class="item_table-header">
<h3 class="title item-description-title" data-marker="item-title"> <a class="item-description-title-link" href="/nizhniy_novgorod/kvartiry/studiya_31.1_m_2324_et._1797799290" itemprop="url" target="_self" title="Студия, 31.1 м², 23/24 эт. в Нижнем Новгороде"> <span itemprop="name">Студия, 31.1 м², 23/24 эт.</span> </a>
</h3>
<div class="about" itemprop="offers" itemscope="" itemtype="http://schema.org/Offer">
<span content="RUB" itemprop="priceCurrency"></span><span class="price price_highlight" content="18300" data-marker="item-price" itemprop="price">
 18 300  <span class="font_arial-rub">₽</span> в месяц  </span> </div>
</div>

In [53]:
commonInfo = flats[0].find('h3', {'class' : 'title item-description-title'}).find('a').find('span').text
commonInfo

'Студия, 31.1 м², 23/24 эт.'

In [54]:
href = flats[0].find('h3', {'class' : 'title item-description-title'}).find('a')['href']
href

'/nizhniy_novgorod/kvartiry/studiya_31.1_m_2324_et._1797799290'

In [58]:
price = flats[0].find('span', {'class' : 'price price_highlight'})['content']
price

'18300'

In [188]:
agency = flats[0].find('div', {'class' : 'data'}).text
agency = re.sub('^\s+|\n|\r|\s+$', '', agency)
agency

'Агентство'

In [69]:
geo = flats[0].find('p', {'class' : 'address'})
geo

<p class="address" data-marker="item-address">
<i class="i-metro i-metro-niz-1" title="Автозаводская линия"></i>
  Чкаловская  <span class="c-2">800 м</span>
  ,    Нижний Новгород, улица Июльских Дней, 1к2  </p>

In [128]:
metroDistance = geo.find('span').text
metroDistance

'800 м'

In [182]:
# metroStation, adress = [t for t in geo.contents if type(t)==NavigableString and t!='\n']
# metroStation = re.sub('^\s+|\n|\r|\s+$', '', metroStation)
# adress = re.sub('^\s+|\n|\r|\s+$', '', adress)
# metroStation, adress
data = geo.contents

list(
    map(lambda x: re.sub('^\s+|\n|\r|\s+$', '', x),
    filter(lambda x: type(x) == NavigableString and x!='\n' ,
           data))
)

['Чкаловская', ',    Нижний Новгород, улица Июльских Дней, 1к2']

In [78]:
metroLine = geo.find('i')['title']
metroLine

'Автозаводская линия'

In [ ]:
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
Разобьем полученные строки на отдельные элементы
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------

In [80]:
commonInfo

'Студия, 31.1 м², 23/24 эт.'

In [125]:
apartmentType, apartmentSquare, floor = commonInfo.split(',')

In [126]:
apartmentType, apartmentSquare, floor

('Студия', ' 31.1 м²', ' 23/24 эт.')

In [127]:
match = re.search(r'\d+.\d+', apartmentSquare)
if match:
    apartmentSquare = match.group()
else:
    apartmentSquare = 'ERROR'
apartmentSquare

'31.1'

In [101]:
apartmentFloor, buildingFloor = re.findall(r'\d+', floor)
apartmentFloor, buildingFloor

('23', '24')

In [ ]:
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
Данные с первой страницы
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------

In [ ]:
def writeErrorTree(tree, pageNumber):
    file = pathToFile("errorTree_%s.txt" % pageNumber, 'errorTrees')
    try:
        with open(file, 'w') as wfile:
            wfile.write(tree)
            logging.info("Error tree has been wrote. Page: %s." % pageNumber)
            print("OK")
    except IOError:
        logging.error("Writing in csv file `%s`!" % csv_file)
        print("I/O error") 

In [8]:
i = 1
url = 'https://www.avito.ru/nizhniy_novgorod/kvartiry/sdam/na_dlitelnyy_srok?p=%s&f=550_5702-5703-5704' % i
response = requests.get(url)

tree = BeautifulSoup(response.content, 'html.parser')
flats = tree.find_all('div', {'class' : 'description item_table-description'})

def getClearApartmentSquare(dirtyApartmentSquare):
    match = re.search(r'\d+.\d+', dirtyApartmentSquare)
    if match:
        clearApartmentSquare = match.group()
    else:
        match = re.search(r'\d+', dirtyApartmentSquare)
        if match:
            clearApartmentSquare = match.group()
        else:
            clearApartmentSquare = 'ERROR'
    return clearApartmentSquare

def getMetroStationAndAdress(data):
    results = list(
            filter(lambda x: x!='' and '</i>' not in x and '</span>' not in x,
                map(lambda x: clearData(x).replace('\xa0', ' '),
                    map(lambda x: str(x).replace('<nobr>', ' ').replace('</nobr>', ' '),
                        data)
                   )
                  )
    )
    metroStation = results[0]
    adress = ' '.join(results[1:])
    return metroStation, adress

def clearData(data):
    return re.sub('^\s+|\n|\r|\s+$', '', data)

def getCommissionPercent(flat):
    commissionPercent = flat.find('span', {'class' : 'about__commission'})
    if commissionPercent == None:
        commissionPercent = '0 %'
    else:
        commissionPercent = clearData(commissionPercent.text)
    return commissionPercent

def getAgency(flat):
    agency = flat.find('div', {'class' : 'data'})
    if agency == None:
        agency = 'None'
    else:
        agency = clearData(agency.text)
    return agency 

def getMetroDistance(geo):
    metroDistance = geo.find('span')
    if metroDistance == None:
        metroDistance = 'None'
    else:
        metroDistance = metroDistance.text
    return metroDistance

def getMetroLine(geo):
    metroLine = geo.find('i')
    if metroLine == None:
        metroLine = 'None'
    else:
        metroLine = metroLine['title']
    return 

#-----------------------------------------------------------------------
#-----------------------------------------------------------------------
#-----------------------------------------------------------------------

flatList = []
for flat in flats:
    try:
        flatDict = {}

        # PART 1
        href = flat.find('h3', {'class' : 'title item-description-title'}).find('a')['href']
        flatDict['href'] = href

        price = flat.find('span', {'class' : 'price'})['content']
        flatDict['price'] = price

        agency = getAgency(flat)
        flatDict['agency'] = agency

        commissionPercent = getCommissionPercent(flat)
        flatDict['commissionPercent'] = commissionPercent

        # PART 2
        geo = flat.find('p', {'class' : 'address'})

        metroDistance = getMetroDistance(geo)
        flatDict['metroDistance'] = metroDistance

        metroStation, adress = getMetroStationAndAdress(geo.contents)
        flatDict['metroStation'] = metroStation
        flatDict['adress'] = adress

        metroLine = getMetroLine(geo)
        flatDict['metroLine'] = metroLine

        # PART 3
        commonInfo = flat.find('h3', {'class' : 'title item-description-title'}).find('a').find('span').text

        apartmentType, apartmentSquare, floor = commonInfo.split(',')
        flatDict['apartmentType'] = apartmentType

        apartmentSquare = getClearApartmentSquare(apartmentSquare)
        flatDict['apartmentSquare'] = apartmentSquare

        apartmentFloor, buildingFloor = re.findall(r'\d+', floor)
        flatDict['apartmentFloor'] = apartmentFloor
        flatDict['buildingFloor'] = buildingFloor

        flatList.append(flatDict)

        logging.info("Handling page `%s`: SUCCESS" % i)
        print("Handling page `%s`: SUCCESS" % i)
    except Exception:
        logging.error("Handling page `%s`: ERROR" % i)
        print("Handling page `%s`: ERROR" % i)


[{'href': '/nizhniy_novgorod/kvartiry/1-k_kvartira_40_m_710_et._1793039317',
  'price': '17000',
  'agency': 'Агентство',
  'commissionPercent': '0 %',
  'metroDistance': '3.6 км',
  'metroStation': 'Буревестник',
  'adress': ',    Нижний Новгород, Союзный проспект, 2',
  'metroLine': None,
  'apartmentType': '1-к квартира',
  'apartmentSquare': '40',
  'apartmentFloor': '7',
  'buildingFloor': '10'},
 {'href': '/nizhniy_novgorod/kvartiry/studiya_31.1_m_2324_et._1797799290',
  'price': '18300',
  'agency': 'Агентство',
  'commissionPercent': '0 %',
  'metroDistance': '800 м',
  'metroStation': 'Чкаловская',
  'adress': ',    Нижний Новгород, улица Июльских Дней, 1к2',
  'metroLine': None,
  'apartmentType': 'Студия',
  'apartmentSquare': '31.1',
  'apartmentFloor': '23',
  'buildingFloor': '24'},
 {'href': '/nizhniy_novgorod/kvartiry/1-k_kvartira_51_m_88_et._1202169600',
  'price': '25000',
  'agency': 'Агентство',
  'commissionPercent': '70 %',
  'metroDistance': '500 м',
  'metroStat

In [10]:
def pathToFile(fileName, subDir = 'data'):
    fileDir = os.path.dirname(os.path.realpath('__file__'))
    return os.path.join(fileDir, subDir, fileName)

def writeToFile(data, oageNumber):
csv_columns = ['href','price','agency', 'commissionPercent', 'metroDistance','metroStation', 'adress', 
               'metroLine', 'apartmentType', 'apartmentSquare', 'apartmentFloor', 'buildingFloor']
csv_file = pathToFile("flats_%s.csv" % i)
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in flatList:
            writer.writerow(data)
        logging.info("Informational message")
        print("OK")
except IOError:
    logging.error("Writing in csv file `%s`!" % csv_file)
    print("I/O error") 

OK


In [ ]:
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
TESTS
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

In [247]:
hardFlat = flats[26].find('p', {'class' : 'address'})
hardFlat.contents

['\n',
 <i class="i-metro i-metro-niz-1" title="Автозаводская линия"></i>,
 '\n  Горьковская  ',
 <span class="c-2">8.1 км</span>,
 '\n  ,    Нижегородская область, Кстовский район, деревня Анкудиновка, жилой ',
 <nobr>комплекс К. М. Анкудиновский</nobr>,
 ' Парк  ']

In [257]:
def getMetroStationAndAdress(data):
    results = list(
            filter(lambda x: x!='' and '</i>' not in x and '</span>' not in x,
                map(lambda x: clearData(x),
                    map(lambda x: str(x).replace('<nobr>', ' ').replace('</nobr>', ' '),
                        data)
                   )
                  )
    )
    metroStation = results[0]
    adress = ' '.join(results[1:])
    return results, adress
getMetroStationAndAdress(hardFlat.contents)

QQQQQQQQ: ,    Нижегородская область, Кстовский район, деревня Анкудиновка, жилой комплекс К. М. Анкудиновский Парк


(['Горьковская',
  ',    Нижегородская область, Кстовский район, деревня Анкудиновка, жилой',
  'комплекс К. М. Анкудиновский',
  'Парк'],
 ',    Нижегородская область, Кстовский район, деревня Анкудиновка, жилой комплекс К. М. Анкудиновский Парк')

In [245]:
'</i>' in '<i class="i-metro i-metro-niz-1" title="Автозаводская линия"></i>' 

False

In [13]:
url = 'https://www.avito.ru/nizhniy_novgorod/kvartiry/sdam/na_dlitelnyy_srok?p=%s&f=550_5702-5703-5704' % 50
response = requests.get(url)
response

<Response [404]>

In [49]:
flatsWhole = tree.findAll('div', {'class' : 'description'})

In [55]:
flatsWhole[5].find('h3', {'class': 'title'})

<h3 class="title item-description-title" data-marker="item-title"> <a class="item-description-title-link" href="/nizhniy_novgorod/kvartiry/2-k_kvartira_50_m_810_et._1696281522" itemprop="url" target="_self" title="2-к квартира, 50 м², 8/10 эт. в Нижнем Новгороде"> <span itemprop="name">2-к квартира, 50 м², 8/10 эт.</span> </a>
</h3>

In [56]:
url = "https://www.avito.ru/nizhniy_novgorod/kvartiry/sdam/na_dlitelnyy_srok"
response = requests.get(url)
tree = BeautifulSoup(response.content, 'html.parser')
flats = tree.find_all('div', {'class' : 'description'})
flats[0]

<div class="description item_table-description"> <div class="item_table-header">
<h3 class="title item-description-title" data-marker="item-title"> <a class="item-description-title-link" href="/nizhniy_novgorod/kvartiry/studiya_31.1_m_2324_et._1797799290" itemprop="url" target="_self" title="Студия, 31.1 м², 23/24 эт. в Нижнем Новгороде"> <span itemprop="name">Студия, 31.1 м², 23/24 эт.</span> </a>
</h3>
<div class="about" itemprop="offers" itemscope="" itemtype="http://schema.org/Offer">
<span content="RUB" itemprop="priceCurrency"></span><span class="price price_highlight" content="15000" data-marker="item-price" itemprop="price">
 15 000  <span class="font_arial-rub">₽</span> в месяц  </span> </div>
</div>
<div class="item_table-extended-contacts-horizontal">
<button class="js-item-extended-contacts button button-origin item-extended-contacts button-origin_small" data-item-id="1797799290">
 Показать телефон
 </button>
</div>
<a class="item_table-extended-description item_table-exten

In [57]:
flats[1].find('div', {'class': 'js-item-date'})

In [ ]:
============================================================================================
============================================================================================
============================================================================================
============================================================================================

In [1]:
path = 'https://yandex.ru/maps/47/nizhny-novgorod/?ll=43.980627%2C56.297119&mode=search&sll=43.957846%2C56.333200&sspn=0.018926%2C0.006163&text=%D0%9D%D0%B8%D0%B6%D0%BD%D0%B8%D0%B9%20%D0%9D%D0%BE%D0%B2%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%20%D0%BF%D1%80%D0%BE%D1%81%D0%BF.%20%D0%93%D0%B0%D0%B3%D0%B0%D1%80%D0%B8%D0%BD%D0%B0%2023%2F9&z=16'

In [9]:
response = requests.get(path)
tree = BeautifulSoup(response.content, 'html.parser')

In [14]:
tree

 <!DOCTYPE html>
 <html lang="ru" prefix="og: http://ogp.me/ns#"> <head> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta charset="utf-8"/> <link href="//yastatic.net/s3/front-maps-static/front-maps//static/v4/icons/favicon/favicon.png" rel="icon" type="image/png"/> <link href="//yastatic.net/s3/front-maps-static/front-maps//static/v4/icons/favicon/favicon.svg" rel="icon" type="image/svg+xml"/> <link color="#FF0000" href="//yastatic.net/s3/front-maps-static/front-maps//static/v4/icons/favicon/safari-pin-icon.svg" rel="mask-icon"/> <link href="//yastatic.net/s3/front-maps-static/front-maps//static/v4/icons/favicon/apple-touch-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/> <meta content="default-src 'self';connect-src 'self' data: blob: wss://push.yandex.ru yandex.ru yandex.com *.yandex.ru *.yandex.com *.yandex.ru *.yandex.net yastatic.net *.yandex-team.ru *.adfox.ru mc.admetrica.ru;frame-src 'self' data: tel: mailto: yandex.ru yandex.ua *.yandex.ru *.yandex.net y

In [15]:
tree.find('input', {'class': 'input_air-search-large__control'})

<input autocomplete="disable" autocorrect="off" class="input_air-search-large__control" placeholder="Поиск мест и адресов" type="text" value="Нижний Новгород просп. Гагарина 23/9"/>

In [49]:
geoCoderUrl = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey=c90067e4-8c6e-4738-8b70-0a0386fa3fdc&geocode=+++,Нижний+Новгород,просп+Гагарина'

In [50]:
responseGeoCoder = requests.get(geoCoderUrl)

In [51]:
import json
dictJson = json.loads(responseGeoCoder.text)

In [87]:
dictJson['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']

'43.97009 56.255327'

In [61]:
geoCoderUrl = 'https://geocode-maps.yandex.ru/1.x/?format=json&apikey=c90067e4-8c6e-4738-8b70-0a0386fa3fdc&geocode=улица+Бекетова,+1Б+Нижний+Новгород'
responseGeoCoder = requests.get(geoCoderUrl)
dictJson2 = json.loads(responseGeoCoder.text)
dictJson2['response']['GeoObjectCollection']

{'metaDataProperty': {'GeocoderResponseMetaData': {'request': 'улица Бекетова, 1Б Нижний Новгород',
   'results': '10',
   'found': '1'}},
 'featureMember': [{'GeoObject': {'metaDataProperty': {'GeocoderMetaData': {'precision': 'exact',
      'text': 'Россия, Нижний Новгород, улица Бекетова, 1Б',
      'kind': 'house',
      'Address': {'country_code': 'RU',
       'formatted': 'Россия, Нижний Новгород, улица Бекетова, 1Б',
       'postal_code': '603057',
       'Components': [{'kind': 'country', 'name': 'Россия'},
        {'kind': 'province', 'name': 'Приволжский федеральный округ'},
        {'kind': 'province', 'name': 'Нижегородская область'},
        {'kind': 'area', 'name': 'городской округ Нижний Новгород'},
        {'kind': 'locality', 'name': 'Нижний Новгород'},
        {'kind': 'street', 'name': 'улица Бекетова'},
        {'kind': 'house', 'name': '1Б'}]},
      'AddressDetails': {'Country': {'AddressLine': 'Россия, Нижний Новгород, улица Бекетова, 1Б',
        'CountryNameCod

In [95]:
commonPart = dictJson2['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']

In [96]:
commonPart['Point']['pos']

'43.985415 56.291184'

In [97]:
commonPart['metaDataProperty']['GeocoderMetaData']['Address']['postal_code']

'603057'